# Einfache Suche im SLSP-Bibliothekskatalog der *Zentralbibliothek Zürich*

Zürich | 5. Mai 2023 - Linda Samsinger

Die Zentralbibliothek Zürich ist eine der grössten Bibliotheken in der Schweiz und stellt eine breite Palette von Sammlungen, darunter Bücher, Zeitschriften, Audio- und Videomaterialien zur Verfügung. Die ZB Zürich hat Zugriff auf den **SLSP-Bibliothekskatalog, welcher Informationen zu allen in Schweizer Bibliotheken verfügbaren Medien enthält**.  

Dieses Jupyter Notebook beinhaltet Textanweisungen und Code-Blöcke zur fachkundigen **Abfrage des SLSP-Katalogs mit Suchbegriffen** und dem **Export des Suchresultats** als Excel, CSV oder JSON-Datei. Es erlaubt Ihnen, gezielt nach Medien zu suchen, welche Ihren spezifischen Interessen und Anforderungen entsprechen. 

Dank diesem Tutorial können **bis zu 10'000 Suchresultate** heruntergeladen werden im Vergleich zur Suchabfrage auf der [Swisscovery-Webseite der UB und ZB Zürich](https://uzb.swisscovery.slsp.ch/discovery/search?vid=41SLSP_UZB:UZB), die es nur erlaubt maximal 50 Suchresultate zu exportieren. Zudem liegen die Suchresultate flach strukturiert und auf einer einzigen Seite übersichtlich vor. 

Über dieses Jupyter Notebook werden nach **Eingabe von Suchbegriffen** im schweizweiten Netzwerk des SLSP-Bibliothekskatalogs  **alle bibliographischen Felder** nach entsprechenden Medien durchsucht. In der **Ausgabe enthalten sind die folgenden Felder**: Titel, Autor, Verlag, Publikationsort, Erscheinungsdatum, Auflage, physische Beschreibung, Sprache, Land, geographisches Thema, Form/Genre, Ressourcentyp, Thema, Zusammenfassung, Epoche, MMS-ID, ISBN, Swisscovery-Link und der Inhaltsverzeichnis-Link, falls vorhanden.   

## Projektidee

### Mit diesem Notebook können Sie...

- **schweizweite Katalogdaten als Grundlage für die akademische Forschungsarbeit beziehen** 
- **Medien der Bibliotheken über Swisscovery-Links einfach bestellen und ausleihen**
- **PDFs über die Inhaltsverzeichnis-Links in der Ergebnisdatei herunterladen**
- **statistische Analysen von Katalogdaten durchführen.**

Für eine weitere Auflistung von Vorteilen für den Benutzer siehe Sektion [Nachnutzung der Ergebnisse](#nachnutzung) weiter unten. 

### Ziel des Notebooks

![resultat](screenshot_suchresultat.png)

![download](screenshot_download.png)

### Quelle: Der SLSP-Bibliothekskatalog 

Die Suchabfrage über den ganzen **SLSP-Bibliothekskatalog** von **über 25 Millionen Einträgen** schweizweit ist über die [Search/Retrieve-URL (SRU)-Schnittstelle (Version 1.2)](https://data.zb.uzh.ch/map/books/data-map-der-zentralbibliothek-zurich/page/alma-sru) der ZB Zürich verfügbar. Der SLSP-Bibliothekskatalog, der mithilfe dieses Jupyter Notebooks abgefragt wird, ist Teil des allgemein genutzten [Swisscovery-Katalogs](https://uzb.swisscovery.slsp.ch/discovery/search?vid=41SLSP_UZB:UZB), welcher allerdings über noch mehr Datenzuflüsse verfügt. 
- Datenquelle: https://data.zb.uzh.ch/map/books/data-map-der-zentralbibliothek-zurich/page/alma-sru
- Datenformat: [MARCXML](https://www.loc.gov/standards/marcxml//)

<a id='konfiguration_der_suchabfrage'></a>
## Konfiguration der Suchabfrage

### Import der Programmabhängigkeiten

Dieses Jupyter Notebook wurde mit Python 3.9.12 erstellt.  

In [23]:
#!python --version

Im folgenden Code-Block werden die benutzten Programmmodule importiert. Bei Bedarf müssen die Programmmodule vorgängig in einer virtuellen Umgebung installiert werden bevor sie importierbar sind. Dazu kommentiert man den dazugehörigen Code ein: 

In [1]:
import piplite
await piplite.install('numpy')
await piplite.install('pandas')
await piplite.install('xmltodict')
await piplite.install('pymarc')
await piplite.install('ipywidgets')
await piplite.install('openpyxl')
await piplite.install('asyncio')

In [2]:
# Module importieren 
import asyncio
import urllib.parse
from io import BytesIO
import micropip
from pyodide.http import open_url, pyfetch
from pathlib import Path
import requests, os
from bs4 import BeautifulSoup as soup
from lxml import etree
import pandas as pd
import xmltodict
import pymarc
from IPython.display import display, HTML, Javascript 
from tqdm import tqdm
import functools as ft


Zusätzlich zu den Python-Modulen braucht es noch folgende Dateien im Jupyter Notebook-Verzeichnis: 
- `screenshot_suchresultat.png`
- `screenshot_download.png`
- `lang_dict.xlsx`
- `country_dict.xlsx`

### Eingabe der Suchkriterien

> Führen Sie den nachstehenden Code-Block aus: "Ctrl" + "Enter" (oder Klick auf das Run-Stopp-Zeichen des Code-Blocks). 

In [3]:
import ipywidgets as widgets 
from ipywidgets import Layout

# Widget "Suchbox" anzeigen 
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-around', 
    width='50%'
    , grid_area='sidebar', align_items = 'center'
)

text = widgets.Text(
    placeholder='z.B. nach "Geschichte alt* Ägypten" im SLSP-Katalog',
    description='Stichwort: ',
    disabled=False
    , layout=form_item_layout
)

display(text)


Text(value='', description='Stichwort: ', layout=Layout(align_items='center', display='flex', flex_flow='row',…

In [26]:
print("Ihr Suchbegriff:\033[1m", text.value, "\033[0m")

Ihr Suchbegriff: Gerstenmalz 


**Geben Sie Ihren Suchbegriff in die Suchbox ein.** Der Suchbegriff wird mit der "Enter"-Taste gespeichert.
<div class="alert alert-block alert-info">
<b>Tipp:</b> Ein möglicher Suchbegriff für den SLSP-Bibliothekskatalog ist der Titel eines Buches, der für Ihre Forschung relevant sein könnte. Ein weiterer möglicher Suchbegriff wäre der Autor eines Buches oder der Artikel, den Sie benötigen. Andere mögliche Suchbegriffe sind Themenbegriffe oder Schlagworte, die mit Ihrem Forschungsinteresse zuammenhängen, oder Erscheinungsjahr und Sprache. Basierend auf dem Suchbegriff wird eine einfache Suchabfrage gestartet, die alle bibliografischen Felder durchsucht. </div>
 

<a id='durchführung_der_suchabfrage'></a>
## Durchführung der Suchabfrage

> **Klick auf "Run All Below"**: Ab dieser Konfigurationsstelle kann der ganze verbleibende Code in einem Zug durchgespielt werden. Klicken Sie dazu auf den darunter stehenden Code-Block, dann im Navigationsmenu auf "Cell" und dann auf "Run All Below", um alle Code-Blöcke auf einmal auszuführen. Die Ausführung kann je nach Anzahl der gefundenen Ergebnisse unterschiedlich lang dauern.  

### Laden von externen Listen und Funktionen

In [6]:
# Sprachen 
df = pd.read_excel(r'lang_dict.xlsx')
lang_dict = dict(zip(df.LangCode, df.LangDe))

In [7]:
def get_lang(lang_value): 
    sprach_value = ''
    for key, value in lang_dict.items(): 
        if key == lang_value: 
            if value == 'None': 
                sprach_value = None 
            else: 
                sprach_value = value 

    return sprach_value 

In [8]:
# Länder
df = pd.read_excel(r'country_dict.xlsx')
ctry_dict = dict(zip(df.CountryCode, df.CountryDe))

In [9]:
def get_country(country_code): 
    country_value = ''
    for key, value in ctry_dict.items(): 
        if key == country_code:
            if value == 'None': 
                country_value = None 
            else: 
                country_value = value 

    return country_value 

In [10]:
# Epoche
def get_epoch(lebens_value):
    if type(lebens_value) == int: 
        if lebens_value <500: 
            epoche_value = 'Antike'
        elif lebens_value <1492: 
            epoche_value = 'Mittelalter'
        elif lebens_value <1914: 
            epoche_value = 'Neuzeit'
        else: 
            epoche_value = 'Gegenwart'
    else: 
        epoche_value = None
    return epoche_value     

In [11]:
# Ressourcentyp 
pubart_dict = {'a': 'Sprachmaterial', 
            'c': 'Noten', 
            'd': 'Noten',
            'e': 'Karte', 
            'f': 'Karte',
            'g': 'Projektionsmedium', 
            'i': 'Tonaufnahme', 
            'j': 'Tonaufnahme',
            'k': 'Grafik', 
            'm': 'Computerdatei',
            'o': 'Satz', 
            'p': 'Gemischt', 
            'r': 'Artefakt',
            't': 'Sprachmaterial'} 

def get_pubart(pubart_value): 
    pbrt_value = ''
    for key, value in pubart_dict.items(): 
        if key == str(pubart_value): 
            pbrt_value = value 

    return pbrt_value 

In [12]:
pubart2_dict = {'a': 'Monographie', 
            'b': 'Zeitschrift', 
            'c': 'Sammlung',
            'd': 'Untereinheit', 
            'i': 'Integrierende Ressource',
            'm': 'Einzeldarstellung', 
            's': 'Zeitschrift'} 

def get_pubart2(pubart2_value): 
    pbrt2_value = ''
    for key, value in pubart2_dict.items(): 
        if key == str(pubart2_value): 
            pbrt2_value = value 

    return pbrt2_value 

In [13]:
def get_pubart3(pubart3_value): 
    pbrt3_value = ''
    if pubart3_value == 'o' or pubart3_value == 'q' or pubart3_value == 's': 
        pbrt3_value = "elektronisch" 
    else: 
        pbrt3_value = "physisch"
    return pbrt3_value   

In [14]:
# wiederholbare Felder 
def get_repeatable_field_value(record, field_no, field_sub, sep=";"): 
        repeatable_field_value = ""
        if len(record.get_fields(field_no)) != 0:
            for el in range(len(record.get_fields(field_no))):
                try:
                    if record.get_fields(field_no)[el][field_sub]!= None:
                        repeatable_field_val = record.get_fields(field_no)[el][field_sub]            
                        repeatable_field_value += sep + repeatable_field_val 
                except: 
                    continue
        else: 
            repeatable_field_value = None 
        try: 
            repeatable_field_value = repeatable_field_value[1:]
        except: 
            repeatable_field_value = None
        return repeatable_field_value

In [15]:
# Feldwerte
def get_record_value(record, field_no, field_sub): 
    record_value = ""
    try: 
        record_value = record.get_fields(field_no)[0][field_sub]
    except: 
        pass
    return record_value 

### Definition der Ausgabefelder

In [16]:
# durchsuchbare Felder
titel_no = '245' #Titel-MARC
titel_sub = 'a' #Titel-MARC (Haupttitel)
titel_sub2 = 'b' #Titel-MARC (Untertitel)
autor_no = '100' #Autor-MARC 
autor_sub = 'a' #Autor-MARC 
autor2_no = '700' #Weitere Autoren-MARC  
autor2_sub1 = 'a' #Weitere Autoren-MARC  
autor2_sub2 = '4' #Weitere Autoren-MARC 
editor_no = '700' #Editoren-MARC 
editor_sub1 = 'a' #Editoren-MARC
editor_sub2 = '4' #Editoren-MARC
editor_sub3 = 'e'  #Editoren-MARC
verlag_no = '264' #Verlag-MARC
verlag_sub = 'b' #Verlag-MARC
pubort_no = '260' #Publikationsort-MARC
pubort_sub = 'a' #Publikationsort-MARC
pubort_no2 = '264' #Publikationsort-MARC
pubort_sub2 = 'a' #Publikationsort-MARC
erschdat_no = '260' #Erscheinungsdatum-MARC
erschdat_sub = 'c' #Erscheinungsdatum-MARC
erschdat_no2 = '264' #Erscheinungsdatum-MARC
erschdat_sub2 = 'c' #Erscheinungsdatum-MARC
erschdat_no3 = '008' #Erscheinungsdatum-MARC
thema_no = '650' #Thema-MARC
thema_sub = 'a' #Thema-MARC
sprache_no = '008' #Sprache-MARC
land_no = '008' #Land-MARC
geo_no = '651' #Geographisches-Thema-MARC
geo_sub = 'a' #Geographisches-Thema-MARC
auflage_no = '250' #Auflage-MARC
auflage_sub = 'a' #Auflage-MARC
zus_no = '520' #Zusammenfassung-MARC
zus_sub = 'a' #Zusammenfassung-MARC
genr_no = '655' #Form/Genre-MARC
genr_sub = 'a' #Form/Genre-MARC
beschr_no = '300' #Beschreibung-MARC
beschr_sub = 'a' #Beschreibung-MARC
pubart3_no = '008' #Ressourcentyp-MARC
authyear_no = '100' #Epoche-MARC
authyear_sub = 'd' #Epoche-MARC
authyear2_no = '700' #Epoche-MARC
authyear2_sub = 'd' #Epoche-MARC

# nichtdurchsuchbare Felder 
mms_id_no = '001' #MMS-ID-MARC
isbn_no = '020' #ISBN-MARC
isbn_sub = 'a' #ISBN-MARC
ilink_no = '856' #Swisscovery-MARC
ilink_sub = 'u' #Swisscovery-MARC

### Öffnen der Suchseite

In [27]:
# Suchbegriff evaluieren
keyword = text.value
replacements = str.maketrans({'ä': 'ae', 'ö': 'oe', 'ü':'ue', 'Ä':'Ae','Ö':'Oe', 'Ü': 'Ue', 'è': 'e', 'é':'e'})
keyword = keyword.translate(replacements)

# URL-Suchabfrage starten 
base_url = "https://slsp-network.alma.exlibrisgroup.com/view/sru/41SLSP_NETWORK?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=all_for_ui%20all%20%22" + keyword + "%22&startRecord=0"
base_url = base_url.replace(" ", "%20")

params = {'recordSchema' : 'marcxml',
      'operation': 'searchRetrieveResponse',
      'version': '1.2'
     }
r = open_url(base_url + "&" +urllib.parse.urlencode(params))
#requests.get(base_url, params=params)

#Suchergebnis analysieren
xml = soup(r.read(), "lxml")
no_records = int(xml.find('numberofrecords').text)

print('Es wurden ', str(no_records), ' Ergebnisse gefunden. Hinweis: Je mehr Ergebnisse vorhanden sind, desto länger dauert das Herunterladen.')


/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Es wurden  5  Ergebnisse gefunden. Hinweis: Je mehr Ergebnisse vorhanden sind, desto länger dauert das Herunterladen.


### Ausführung der Suche

In [29]:
# Seitenindex
timeout_page = 0 # die Zahl ersetzen, falls das Herunterladen unterbrochen wurde, siehe #print(nextpage)
if timeout_page > 0: 
    timeout_page += 1 
nextpage = 0 + timeout_page
results_dict = {}

# Ladebalken 
pbar = tqdm(total = int(no_records))

filename = "website.xml"
p = Path(filename)
# Iteration über alle Webseiten der URL mit den Suchergebnissen 
while nextpage < no_records:
    next_url = base_url[:-1] + str(nextpage + timeout_page)
    response = open_url(next_url)
    response_content = response.read()
    try: 
        os.unlink(filename)
    except: 
        pass
    p.write_text(response_content)
    dict_data = xmltodict.parse(response_content)
    records = pymarc.parse_xml_to_array(p.name, strict=True)

    for i, record in enumerate(records):
        try: 
            index = int(dict_data['searchRetrieveResponse']['records']['record']['recordPosition'])
        except: 
            index = int(dict_data['searchRetrieveResponse']['records']['record'][i]['recordPosition'])
            
        # durchsuchbare Felder:    

        # Titel 
        titel_value = ""
        if len(record.get_fields(titel_no)) != 0: 
            titel_value = get_record_value(record, titel_no, titel_sub)  
            titel_value2 = get_record_value(record, titel_no, titel_sub2)
            if titel_value2: 
                titel_value = ', '.join(filter(None, [titel_value, titel_value2]))
        else: 
            titel_value = None 
        if titel_value != None and len(titel_value) > 1:
            titel_value = titel_value.replace("[", "")
            titel_value = titel_value.replace("]", "")
            titel_value = titel_value.replace("<<", "")
            titel_value = titel_value.replace(">>", "")
            if titel_value[-1] == "/" or titel_value[-1] == ".": 
                titel_value = titel_value[:-1]
        
        # Autor
        autor_value = ""
        vorname_value = ""
        nachname_value = ""
        if len(record.get_fields(autor_no)) != 0: 
            autor_value = get_record_value(record, autor_no, autor_sub)              
        else: 
            autor_value = None
        if autor_value != None and len(autor_value) > 1:  
            if autor_value[-1] == "," or (autor_value[-1] == "." and not type(autor_value[-2]) == str): 
                autor_value = autor_value[:-1]
            autor_value = autor_value.strip()    
            autor_lst = autor_value.split(',')
            try: 
                vorname_value, nachname_value = autor_lst[1].strip(), autor_lst[0].strip()
            except: 
                nachname_value = autor_lst[0]
                vorname_value = None
        
        # Weitere Autoren
        autor2_values = ""
        autor2_lst2 = []
        if len(record.get_fields(autor2_no)) != 0: 
            for el in range(len(record.get_fields(autor2_no))):
                if get_record_value(record, autor2_no, autor2_sub2)  == 'aut': 
                    autor2_value = record.get_fields(autor2_no)[el][autor2_sub1]   
                    autor2_values += "; " +  autor2_value 
        else: 
            autor2_values = None
        if autor2_values!= None and len(autor2_values) > 1: 
            if autor2_values[0] == ";": 
                autor2_values = autor2_values[1:].strip()
            autor2_lst = [ el.split(",") for el in [el.strip()  for el in autor2_values.split(';') if el != '']] 
            for el in autor2_lst:
                try: 
                    el[0], el[1] = el[1], el[0]
                    autor2_names = el[0].strip() + ' ' + el[1].strip()
                    autor2_lst2.append(autor2_names)
                except: 
                    autor2_lst2.append(el[0])
            autor2_values =  '; '.join(autor2_lst2) 

        
        # Editoren
        editor_values = ""
        editor_lst2 = []
        if len(record.get_fields(editor_no)) != 0: 
            for el in range(len(record.get_fields(editor_no))):
                if get_record_value(record, editor_no, editor_sub2)  == 'edt': 
                    editor_value = record.get_fields(editor_no)[el][editor_sub1]   
                    editor_values += "; " +  editor_value 
                try: 
                    if 'editor' in get_record_value(record, editor_no, editor_sub3): 
                        editor_value = record.get_fields(editor_no)[el][editor_sub1]   
                        editor_values += "; " +  editor_value 
                except: 
                    continue
        else: 
            editor_values = None
        if editor_values!= None and len(editor_values) > 1: 
            if editor_values[0] == ";": 
                editor_values = editor_values[1:].strip()
            editor_lst = [el.split(",") for el in [el.strip() for el in editor_values.split(';') if el != '']]
            for el in editor_lst: 
                try: 
                    el[0], el[1] = el[1], el[0]
                    editor_lst = el[0].strip() + ' ' + el[1].strip()
                    editor_lst2.append(editor_lst)
                except: 
                    editor_lst2.append(el[0])
                editor_values =  '; '.join(editor_lst2)
    
        # Verlag
        verlag_value = get_repeatable_field_value(record, verlag_no, verlag_sub)
        if verlag_value != None and len(verlag_value) > 1:
            if verlag_value[-1] == "," or verlag_value[-1] == ":": 
                verlag_value = verlag_value[:-1]
            verlag_value = verlag_value.replace("[", "")
            verlag_value = verlag_value.replace("]", "")
            verlag_value = verlag_value.replace("?", "")
            verlag_value = verlag_value.strip() 
            
        # Publikationsort
        pubort_value = ""
        if len(record.get_fields(pubort_no)) != 0:
            for el in range(len(record.get_fields(pubort_no))):
                try: 
                    if record.get_fields(pubort_no)[el][pubort_sub]!= None:
                        pubort_val = record.get_fields(pubort_no)[el][pubort_sub]            
                        pubort_value += ";" + pubort_val   
                except: 
                    continue
        else: 
            pubort_value = ""
            if len(record.get_fields(pubort_no2)) != 0:
                for el in range(len(record.get_fields(pubort_no2))):
                    if get_record_value(record, pubort_no2, pubort_sub2)!= None:
                        pubort_val = get_record_value(record, pubort_no2, pubort_sub2)           
                        pubort_value += ";" + pubort_val     
            else: 
                pubort_value = None 
            try: 
                pubort_value = pubort_value[1:]
            except: 
                pubort_value = None 
        if pubort_value != None and len(pubort_value) > 1:
            if pubort_value[0] == ";" or pubort_value[0] == "," : 
                pubort_value = pubort_value[1:]
            if pubort_value[-1] == ":" or pubort_value[-1] == ";": 
                pubort_value = pubort_value[:-1]
            if type(pubort_value) == int: 
                pubort_value = None 
            pubort_value = pubort_value.replace("[", "")
            pubort_value = pubort_value.replace("]", "")
            pubort_value = pubort_value.replace("?", "")
            pubort_value = pubort_value.strip()
            if pubort_value.isdigit():
                pubort_value = None
      
         # Erscheinungsjahr
        erschdat_value = ""
        if len(record.get_fields(erschdat_no)) != 0:
            for el in range(len(record.get_fields(erschdat_no))):
                try: 
                    if record.get_fields(erschdat_no)[el][erschdat_sub]!= None:
                        erschdat_val = record.get_fields(erschdat_no)[el][erschdat_sub]            
                        erschdat_value += ";" + erschdat_val 
                except: 
                    continue
        else: 
            erschdat_value = ""
            if len(record.get_fields(erschdat_no2)) != 0:
                for el in range(len(record.get_fields(erschdat_no2))):
                    if get_record_value(record, erschdat_no2, erschdat_sub2)!= None:
                        erschdat_val = get_record_value(record, erschdat_no2, erschdat_sub2)            
                        try: 
                            if erschdat_val[-1] == ".": 
                                erschdat_val = erschdat_val[:-1]
                            erschdat_value += ";" + erschdat_val 
                        except: 
                            continue
            else: 
                if len(record.get_fields(erschdat_no3)) != 0:
                    erschdat_value = record.get_fields(erschdat_no3)[0].value()[6:11]            
                else: 
                    erschdat_value = None 
        if erschdat_value != None and len(erschdat_value) > 1:  
            if erschdat_value[0] == ";": 
                erschdat_value = erschdat_value[1:]
            if erschdat_value[-1] == ".": 
                erschdat_value = erschdat_value[:-1]
            erschdat_value = erschdat_value.replace("[", "")
            erschdat_value = erschdat_value.replace("]", "")
            erschdat_value = erschdat_value.replace("©", "")
            erschdat_value = erschdat_value.replace("℗", "")
            erschdat_value = erschdat_value.replace("?", "")
            erschdat_lst = erschdat_value.split(";")
            erschdat_lst = [el.strip() for el in erschdat_lst] 
            for i, el in enumerate(erschdat_lst):  
                try: 
                    if el[1].isdigit() and el[0].isalpha(): 
                        el = el.replace(el[0], "")
                        erschdat_lst[i] = el
                except: 
                    continue
            erschdat_value = ';'.join(list(set(erschdat_lst)))
            erschdat_value = erschdat_value.strip()
        try: 
            if 'X' in erschdat_value or 'I' in erschdat_value or 'M' in erschdat_value or 'L' in erschdat_value or 'V' in erschdat_value: 
                erschdat_value = erschdat_value
            else: 
                if 'J' not in erschdat_value: 
                    erschdat_lst = erschdat_value.split(" ")
                    erschdat_lst = [el for el in erschdat_lst if el.isdigit() or el[0].isdigit() or el[1].isdigit() or el[2].isdigit()]
                    erschdat_value = ';'.join(list(dict.fromkeys(erschdat_lst)))
                else: 
                    erschdat_value = erschdat_value
        except: 
            pass   
            
        # Auflage   
        auflage_value = get_repeatable_field_value(record, auflage_no, auflage_sub)
        if auflage_value != None and len(auflage_value) > 1: 
            auflage_value = auflage_value.replace("[", "")
            auflage_value = auflage_value.replace("]", "")
        
        # Physische Beschreibung
        beschr_value = get_repeatable_field_value(record, beschr_no, beschr_sub)
            
        # Sprache
        if len(record.get_fields(sprache_no)) != 0:
            lang_value = record.get_fields(sprache_no)[0].value()[35:38] 
            if get_lang(lang_value) or get_lang(lang_value) == None: 
                lang_value = get_lang(lang_value)
        else: 
            lang_value = None 
            
        # Land
        if len(record.get_fields(land_no)) != 0:
            ctry_value = record.get_fields(land_no)[0].value()[15:18]
            ctry_value = ctry_value.strip()
            if get_country(ctry_value) or get_country(ctry_value) == None: 
                ctry_value = get_country(ctry_value)
        else: 
            ctry_value = None 
        
        # Geographisches Thema
        geo_value = get_repeatable_field_value(record, geo_no, geo_sub)      
        if geo_value != None and len(geo_value) > 1: 
            geo_value = geo_value.replace("[", "")
            geo_value = geo_value.replace("]", "")
            if geo_value[-1] == "." or geo_value[-1] == ";": 
                geo_value = geo_value[:-1]
            geo_value = geo_value.strip()
            
        # Form/Genre
        genr_value = get_repeatable_field_value(record, genr_no, genr_sub)
        if genr_value != None and len(genr_value) > 1:  
            if genr_value[0] == ";": 
                genr_value = genr_value[1:]
            if genr_value[-1] == ".": 
                genr_value = genr_value[:-1]
            genr_value = genr_value.replace("[", "")
            genr_value = genr_value.replace("]", "")
            genr_lst = genr_value.split(";")
            genr_lst = [el.strip() for el in genr_lst]
            genr_value = ';'.join(list(set(genr_lst)))
            genr_value = genr_value.strip()     
        
        #Ressourcentyp 
        if len(record.leader) != 0:
            pubart_value = record.leader[6] 
            if get_pubart(pubart_value) or get_pubart(pubart_value) == None: 
                pubart_value = get_pubart(pubart_value)
        else: 
            pubart_value = None 
     
        if len(record.leader) != 0:
            pubart2_value = record.leader[7] 
            if get_pubart2(pubart2_value) or get_pubart2(pubart2_value) == None: 
                pubart2_value = get_pubart2(pubart2_value)
        else: 
            pubart2_value = None 
   
        if len(record.get_fields(pubart3_no)) != 0:
            try: 
                pubart3_value = record.get_fields(pubart3_no)[0].value()[23]
            except: 
                pubart3_value = record.get_fields(pubart3_no)[0].value()[29]
            if get_pubart3(pubart3_value) or get_pubart3(pubart3_value) == None: 
                pubart3_value = get_pubart3(pubart3_value)
        else: 
            pubart3_value = None 
              
        if pubart_value == 'Sprachmaterial' and (pubart2_value == 'Einzeldarstellung' or pubart2_value == 'Untereinheit' or pubart2_value == 'Sammlung' or pubart2_value == 'Monographie'): 
            publart_value = 'Buch' + ' - ' + pubart3_value
        elif pubart_value == 'Sprachmaterial' and (pubart2_value == 'Integrierende Ressource' or pubart2_value == 'Zeitschrift'): 
            publart_value = 'Zeitschrift'  + ' - ' + pubart3_value
        elif (pubart_value == 'Gemischt' or pubart_value == 'Satz' or pubart_value == 'Artefakt'): 
            publart_value = 'Gemischtes Material' + ' - ' + pubart3_value
        elif pubart_value != 'Sprachmaterial' or pubart_value != 'Gemischt' or pubart_value != 'Satz' or pubart_value != 'Artefakt':    
            publart_value = pubart_value + ' - ' + pubart3_value
        else: 
            publart_value = None

            
        # Thema  
        thema_value = get_repeatable_field_value(record, thema_no, thema_sub, sep=" ") 
        if thema_value != None and len(thema_value) > 1: 
            thema_lst = thema_value.split(" ")
            thema_lst = [el.strip() for el in thema_lst if el != '']
            thema_value = ' '.join(list(dict.fromkeys(thema_lst)))
            thema_value = thema_value.strip()
        
        # Zusammenfassung
        zus_value = get_repeatable_field_value(record, zus_no, zus_sub, sep=" ") 
         
        
        # Epoche
        epoche_value = ""
        epoche_all = ""
        try:
            authyear_value = record.get_fields(authyear_no)[0][authyear_sub] 
            if authyear_value != None: 
                authyear_value = authyear_value.replace("-", "")
                if len(authyear_value) == 4: 
                    try: 
                        authyear_value = int(authyear_value)
                        epoche_value = get_epoch(authyear_value)
                    except: 
                        epoche_value = None 
                elif len(authyear_value) == 8: 
                    try: 
                        authyear_value = int(authyear_value[:4])
                        epoche_value = get_epoch(authyear_value)
                    except: 
                        authyear_value = int(authyear_value[4:])
                        epoche_value = get_epoch(authyear_value) 
                else: 
                    epoche_value = None
            else: 
                epoche_value = None
        except:
            epoche_all = ""
            for el in range(len(record.get_fields(authyear2_no))):
                if get_record_value(record, authyear2_no, authyear2_sub)!= None:
                    authyear2_val = get_record_value(record, authyear2_no, authyear2_sub)  
                    authyear2_value = authyear2_val.replace("-", "")
                    if len(authyear2_value) == 4: 
                        try: 
                            authyear2_value = int(authyear2_value)
                            epoche_value = get_epoch(authyear2_value)
                        except: 
                            epoche_value = None 
                    elif len(authyear2_value) == 8: 
                        try: 
                            authyear2_value = int(authyear2_value[:4])
                            epoche_value = get_epoch(authyear2_value)
                        except: 
                            try: 
                                authyear2_value = int(authyear2_value[4:])
                                epoche_value = get_epoch(authyear2_value) 
                            except: 
                                epoche_value = None 
                    else: 
                        epoche_value = None 
                    try: 
                        epoche_all += ";" +  epoche_value 
                    except: 
                        continue

        if epoche_all: 
            epoche_all = epoche_all[1:] 
            epoche_lst = epoche_all.split(";")
            epoche_lst = [el.strip() for el in epoche_lst]
            epoche_value = ';'.join(list(dict.fromkeys(epoche_lst)))
            epoche_value = epoche_value.strip() 


        # nicht durchsuchbare Felder:      
        # MMS-ID & Swisscovery-Link
        if len(record.get_fields(mms_id_no)) != 0:
            mms_id_value = record.get_fields(mms_id_no)[0].value()
            slink_value = "https://uzb.swisscovery.slsp.ch/permalink/41SLSP_UZB/1d8t6qj/alma" + str(mms_id_value)
        else: 
            mms_id_value = None
            slink_value = None
        
        # ISBN
        isbn_value = get_repeatable_field_value(record, isbn_no, isbn_sub)
        
        # Inhaltsverzeichnis-Link
        ilink_value = get_repeatable_field_value(record, ilink_no, ilink_sub)
        
        
        # Resultate: 
        results_dict[index] = (titel_value, vorname_value, nachname_value, autor2_values, editor_values, verlag_value, pubort_value, erschdat_value, auflage_value, beschr_value, lang_value, ctry_value, geo_value, genr_value, publart_value, thema_value, zus_value, epoche_value, mms_id_value, isbn_value, slink_value, ilink_value)
        #print(index)
        pbar.update(1)

    if 'nextRecordPosition' in dict_data['searchRetrieveResponse'].keys(): 
        nextpage = dict_data['searchRetrieveResponse']['nextRecordPosition']
        nextpage = int(nextpage) + timeout_page
        #print(nextpage)
    else: 
        break 

pbar.close()    


  0%|          | 0/5 [00:35<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 34.25it/s]


**Warten Sie bis der Ladebalken vollständig geladen hat.** <br>
<div class="alert alert-block alert-danger">
<b>Warnung:</b> Falls das Herunterladen wegen Verbindungsproblemen unterbrochen wurde, so kann man den Code unterhalb auskommentieren und im oberen Code-Block die Variable `timeout_page` mit dem Wert der Ausgabe des Code-Blocks unten ersetzen, bevor man den oberen Code-Block erneut ausführt. So kann man die Suche weiterführen ohne von vorn beginnen zu müssen.
</div> 

In [ ]:
#print(nextpage)

<a id='strukturierung_des_suchresultats'></a>
## Strukturierung des Suchresultats


Die Suchergebnisse werden tabellarisch mit den Ausgabefeldern als Spaltenname dargestellt. Die Suchergebnisse enthalten dadurch detaillierte Informationen zu den einzelnen Werken. Dies erleichtert es, eine fundierte Entscheidung über die Medien zu treffen. 

In [30]:
# Suchergebnis als Tabelle
df = pd.DataFrame.from_dict(results_dict, orient='index')

#Spaltennamen 
df.columns = [['Titel', 'Autor', '', 'Weitere Autoren', 'Editoren', 'Verlag', 'Publikationsort', 'Erscheinungsdatum', 'Auflage', 'Physische Beschreibung', 'Sprache', 'Land', 'Geographisches Thema', 'Form/Genre', 'Ressourcentyp', 'Thema', 'Zusammenfassung', 'Epoche', 'MMS-ID', 'ISBN', 'Swisscovery-Link', 'Inhaltsverzeichnis-Link'], 
              ['', 'Vorname', 'Nachname', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
#Reihenindex
df.index.name = 'ID'
df.index += 1 
df

,Titel,Autor,,Weitere Autoren,Editoren,Verlag,Publikationsort,Erscheinungsdatum,Auflage,Physische Beschreibung,...,Geographisches Thema,Form/Genre,Ressourcentyp,Thema,Zusammenfassung,Epoche,MMS-ID,ISBN,Swisscovery-Link,Inhaltsverzeichnis-Link
,,Vorname,Nachname,,,,,,,,...,,,,,,,,,,
ID,,,,,,,,,,,,,,,,,,,,,
1,"6.000 Jahre altes Gerstenmalz aus Hornstaad, B...",Renate,Ebersbach,Elena Marinova; Andreas G. Heiss,,None,None,2020,None,None,...,None,None,Buch - physisch,None,Bodensee - Malz - Bierherstellung - AOV,Gegenwart,991170872816905501,None,https://uzb.swisscovery.slsp.ch/permalink/41SL...,None
2,Die Bierbrauerei,Ludwig,Narziss,,,None,Weinheim,2009,"8., überarbeitete und erg. Aufl. /",1 online resource (822 p.),...,None,None,Buch - elektronisch,Beer. Brewing.,"Die lang erwartete, vollständig überarbeitet...",Gegenwart,991170318083805501,1-282-31445-9;9786612314452;3-527-62863-0;3-52...,https://uzb.swisscovery.slsp.ch/permalink/41SL...,None
3,Proanthocyanidine in den Braurohstoffen Gerste...,Wolfgang,Friedrich,None,None,,Bonn,2000,None,183 S.,...,None,Hochschulschrift,Buch - physisch,Braugerste HPLC-MS Malzextrakt Proanthocyanidi...,None,,991116664539705501,None,https://uzb.swisscovery.slsp.ch/permalink/41SL...,None
4,Ueber den Eiweissabbau bei der Autolyse von Ge...,Rudolf,Buse,None,None,Postberg,Bottrop i.W.,1933,None,54 S.,...,None,Hochschulschrift,Buch - physisch,None,None,,991106171499705501,None,https://uzb.swisscovery.slsp.ch/permalink/41SL...,None
5,Über drei eigenartige Erstickungstodesfälle du...,Anton,Hufnagl,None,None,Mayr,Würzburg,1939,None,30 S.,...,None,Hochschulschrift,Buch - physisch,None,None,,991021502469705501,None,https://uzb.swisscovery.slsp.ch/permalink/41SL...,None


<a id='exportieren_des_suchresultats'></a>
## Exportieren des Suchresultats

Das Suchresultat kann als Excel, CSV oder JSON exportiert werden. 

In [31]:
df.to_excel("ZB_Suchresultat.xlsx", sheet_name='Resultat')  

In [ ]:
#df.to_csv("ZB_Suchresultat.csv", index=False, sep=';')

In [ ]:
#df.to_json('ZB_Suchresultat.json', orient="index")

**Jetzt kann die Ergebnisdatei im Verzeichnis dieses Jupyter Notebooks geöffnet werden.**

<div class="alert alert-block alert-info">
<b>Tipp:</b> Sie können die Ergebnisse weiter filtern z.B. nach Sprache, Autor, Thema oder anderen Kriterien. Die Suchergebnisse können auch sortiert werden, z.B. alphabetisch nach Autor oder nach Erscheinungsjahr. Dies erhöht die Chance, dass die gewünschten Materialien gefunden werden. Klicken Sie auf den Swisscovery-Link, um weitere Details zu sehen wie Bibliotheksstandort, Signatur und Verfügbarkeit des Mediums.  </div>

### Optionaler Zusatz: Download der Inhaltsverzeichnisse

Die Inhaltsverzeichnisse zu jedem Medium können dank dem Inhaltsverzeichnislink in der Tabelle auf den lokalen Computer als Datei heruntergeladen werden, falls so ein Link zum Medium existiert. 

>Kommentieren Sie den untenstehen Code-Block aus (Wegnehmen der zwei Mal drei Gänsefüsschen am Anfang und Ende) und geben Sie den Speicherort `save_to_path` der zu herunterladenden Inhaltsverzeichnisse an, bevor Sie ihn ausführen. Ansonsten werden die Inhaltsverzeichnisse als PDF-Dateien im aktuellen Verzeichnis (das des Jupyter Notebooks) abgespeichert.  

In [ ]:
'''
ihv_dict = {}
for i, row in df.iterrows(): 
    if row['Inhaltsverzeichnis-Link'][0] != None:
        titel_cap = []
        for el in row['Titel', ''].split(" "): 
            el = el[0].upper() + el[1:]
            titel_cap.append(el)
        titel_cap = ' '.join(titel_cap)
        if len(row['Inhaltsverzeichnis-Link'][0].split(";")) == 1: 
            filename = f"{i}_{row['', 'Nachname']}_{row['Erscheinungsdatum', '']}_{titel_cap}"
            remove_chars = "[/:*?\"<>|],'- "
            for char in remove_chars: 
                filename = filename.replace(char, "")
            #filename = filename[:31]
            ihv_dict[filename] = row['Inhaltsverzeichnis-Link'][0]
        else: 
            for im, ihv_link in enumerate(list(row['Inhaltsverzeichnis-Link'][0].split(";"))): 
                filename = f"{i}.{im+1}_{row['', 'Nachname']}_{row['Erscheinungsdatum', '']}_{row['Titel', '']}"
                remove_chars = "[/:*?\"<>|],'- "
                for char in remove_chars: 
                    filename = filename.replace(char, "")
                #filename = filename[:31]
                ihv_dict[filename] = ihv_link

print('Es wurden ', len(ihv_dict), ' Links fürs Herunterladen gefunden.')
'''

In [ ]:
'''
save_to_path = None
#save_to_path = r"C:/Benutzer"
i = 0
for filename, link in ihv_dict.items(): 
    try: 
        response = requests.get(link)
        if not save_to_path: 
            complete_path = os.path.expanduser(f"~/downloads/{filename}.pdf")
        else: 
            complete_path = os.path.join(f"{save_to_path}/{filename}.pdf")
        with open(complete_path, 'wb') as f: 
            f.write(response.content)
        i += 1
    except: 
        continue

print('Es wurden ', i, ' von ', len(ihv_dict), ' Links heruntergeladen.')
'''

<a id='nachnutzung'></a>
### Nachnutzung der Ergebnisse

- **Erstellung einer Bibliographie (= Liste von Referenzen)**

Sie möchten alle Referenzdaten der Bücher mit einem bestimmten Suchbegriff (Autor, Titel, Verlag, Publikationsort, Erscheinungsdatum) herunterladen, z.B. für die Erstellung einer Bibliographie einer (akademischen) **Forschungsarbeit** oder zur Literaturauswertung zu Beginn einer Forschungsarbeit. Dazu braucht es Rohdaten, um diese einem gewissen Zitationsstil anpassen zu können. Auch Literaturverwaltungssysteme wie Citavi, Endnote, Mendeley und Zotero können mit den Daten eingepflegt werden.  
- **Zugriff auf Swisscovery-Links für Bestellungen**

In der Ergebnisdatei findet sich mitunter eine Spalte "Swisscovery-Link". Der Nutzer kann die [Swisscovery-Links](https://uzb.swisscovery.slsp.ch/discovery/search?vid=41SLSP_UZB:UZB) nutzen, um **Bücher vorbestellen bzw. reservieren** zu können. Dank den Swisscovery-Links können die Medien auf der Hauptseite der Bibliothek und weitere dazugehörige Informationen ausfindig gemacht werden; deren Standort (Bibliothek, Geschoss, Magazin), Signatur und Ausleihinformationen sind auf Swisscovery verfügbar. Die Swisscovery-Seite bietet auch noch weitere Filtermöglichkeiten an zur Einschränkung der Suche.  

- **Überblick über alle Zusammenfassungen**

In der Ergebnisdatei gibt es eine Spalte, die "Zusammenfassung" heisst. Der Nutzer kann die Zusammenfassungen/Abstracts eines Suchbegriffs dadurch einsehen. Eine Zusammenfassung bietet einen Überblick über das entsprechende Buch. Sie kann dazu dienen, zu verstehen, ob das Buch wirklich zum **Forschungsthema** passt, oder ob man es verwerfen kann.    
- **Inhaltsverzeichnisse als PDF-Dateien**

Die Ergebnisdatei beinhaltet die Spalte "Inhaltsverzeichnis-Link". Der Nutzer kann dank den Links alle Inhaltsverzeichnisse als PDF herunterladen, damit man bei mehreren Autoren eines Buches z.B. die **Autorenliste** im Inhaltsverzeichnis überblicken kann.  Mit der Spalte "Inhaltsverzeichnis-Links" können die Links zu den PDF-Inhaltsverzeichnissen mit ein paar Zeilen Code direkt lokal auf den Computer heruntergeladen werden. Die Dateinamen der PDF-Inhaltsverzeichnisse sind nach Index, Nachname des Autors, Erscheinungsjahr und abgekürzter Titel des Medium benannt.  

- **Meta-Statistiken**

Der Nutzer kann die detaillierten Angaben in der Ergebnisdatei als Suchresultat des Bibliothekskatalogs dazu nutzen, um damit  **statistische Analysen** der bibliographischen (Meta-)Daten durchzuführen. Z.B. kann nach Anzahl, Auflage, Sprache, Genre, Epoche und Thema analytisch erhoben werden. Man kann die Suchresultate mit geeigneten statistischen Diagrammen visuell anschaulich auswerten wie zum Beispiel Balken-, Kuchen- oder Liniendiagramme.  

- **Einbettung in Websites oder Web-Apps**

Der Nutzer möchte das Suchresultat im JSON-Format in Websites oder Web-Apps einbetten.

Mit den oben vorgeführten Schritten dieses Notebooks können Sie eine einfache Suchabfrage des SLSP-Bibliothekskatalogs durchführen, um den Bibliothekskatalog optimal zu nutzen und Medien zu finden, die Ihrem spezifischen Forschungsinteresse entsprechen. Dieses Jupyter Notebook bietet dazu eine optimierte und effiziente Möglichkeit, nach Medien zu suchen, welche die individuellen Anforderungen erfüllen. 

*Für weitere Fragen und Feedback wenden Sie sich bitte an Linda Samsinger, Metadaten-Expertin an der Zentralbibliothek Zürich, unter linda.samsinger@zb.uzh.ch.* 

**Zentralbibliothek Zürch** | Kantons-, Stadt- und Universitätsbibliothek 

Folgen Sie uns auf sozialen Medien:

<nav>
<a href="https://www.facebook.com/Zentralbibliothek.Zuerich/?locale=de_DE">Facebook</a> |
<a href="https://twitter.com/zbzuerich?lang=de">Twitter</a> |
<a href="https://www.youtube.com/channel/UCVLFfTBvY89xBI_rEBgmdvQ">YouTube</a> |
<a href="https://www.instagram.com/zentralbibliothek_zuerich/?hl=de">Instagram</a> |
<a href="https://ch.linkedin.com/company/zentralbibliothek-z%C3%BCrich">LinkedIn</a>  
</nav>